In [1]:
import pandas as pd
from utils import *

In [16]:
# Split each category to its own CSV file from files the crawler created
categories_list = []

for i in range(0,4):
    df = pd.read_csv(f"data/reviews_part_1.{i}.csv")
    for cat in df.Category.unique():
        if cat not in categories_list: 
            categories_list.append(cat)
        cat_df = df[df['Category'] == cat]
        # print(cat)
        # Append_Or_Create_CSV_By_Name(f"{cat}.csv", cat_df)

print(categories_list)

C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\2452893455.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/reviews_part_1.{i}.csv")
C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\2452893455.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/reviews_part_1.{i}.csv")


['דודי שמש וחשמל', 'שרותי מנעולנים', 'שרותי הובלות', 'התקנה ותיקון מזגנים']


In [18]:
# Dropping duplicated reviews. (in the website a review is visible in all its sub-categories so we drop those duplications)
for cat in categories_list:
    df = pd.read_csv(f"data/{cat}.csv")
    df.drop_duplicates(subset=['Date','Customer Name','Review', 'Professional SysID'], keep='first', inplace=True)
    Save_CSV(cat+".csv", df)

C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\3192907318.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/{cat}.csv")


In [ ]:
# Merge Same Categories CSV's 

df = pd.read_csv("data\\התקנה ותיקון מזגנים.csv")
df2 = pd.read_csv("data\\AirConditioners.csv")

frames = [df, df2]

df3 = pd.concat(frames)
df3_new = df3.drop_duplicates(subset=['Date','Customer Name','Review', 'Professional SysID'],keep='first')

Save_CSV('התקנה ותיקון מזגנים.csv', df3_new)

print(f'{df3.shape[0]}, {df3.shape[0]}')


In [74]:
# Add 'Year' Column
list = Get_All_CSV_Names()

for cat in list:
    df = pd.read_csv(f"data/{cat}.csv") 
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    Save_CSV(f"{cat}.csv", df)

C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\3052088807.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/{cat}.csv")


In [3]:
# Add region to each review (across all categories) from a CSV file we downloaded from "https://data.gov.il/dataset/citiesandsettelments"
# if the "Location" includes digits are null we set the region as "חסוי" otherwise we set the region based on the CSV we downloaded

filenames = Get_All_CSV_Names()
cities = pd.read_csv("data/map/israel_towns.csv")
not_found = []

for csv in filenames:
    print(csv)
    df = pd.read_csv(f"data/{csv}.csv")
    changes = 0

    df["Region"] = None
    for i, row in df.iterrows():
        city = row["Location"]
        if re.search(r"[\d]+", str(city)) or pd.isna(city):
            df.at[i, "Location"] = "חסוי"
            df.at[i, "Region"] = "חסוי"
            changes += 1
            # print(city)
        else:
            area = cities[cities["Name"] == city]["District"].values
            if area.shape[0] > 0:
                df.at[i, "Region"] = area[0]
            else:
                not_found.append(city)
    Save_CSV(f"{csv}.CSV", df)
    print(not_found)
    print(changes)
    # break


שרותי הובלות


C:\Users\Omer\AppData\Local\Temp\ipykernel_16848\292625568.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/{csv}.csv")


[]
1780
התקנה ותיקון מזגנים
[]
2057
דודי שמש וחשמל
[]
476
שרותי אינסטלציה
[]
1465
מוסכים
[]
1189
שרותי מנעולנים
[]
621


In [23]:
# Creating "Is Hidden" column if the "Customer Name" is not clearly visible (ex: names with first letter of first and last names or random digits)
filenames = Get_All_CSV_Names()

for csv in filenames:
    print(csv)
    df = pd.read_csv(f"data/{csv}.csv")
    df["Is Hidden"] = 0
    df["Customer Name"].fillna(value="חסוי", inplace=True)
    df.loc[df["Customer Name"].str.contains("[א-ת]\.\s?[א-ת]", regex=True), "Is Hidden"] = 1
    df.loc[df["Customer Name"].str.contains("\d+", regex=True), "Is Hidden"] = 1
    df.loc[df["Customer Name"].str.contains("חסוי", regex=True), "Is Hidden"] = 1
    Save_CSV(f"{csv}.CSV", df)
# df2 = df.loc[df["Customer Name"].str.contains("[א-ת]\.\s?[א-ת]", regex=True)]
# print(pd.unique(df2["Is Hidden"]))

    


שרותי הובלות


C:\Users\Omer\AppData\Local\Temp\ipykernel_2452\1123147818.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/{csv}.csv")


התקנה ותיקון מזגנים
דודי שמש וחשמל
שרותי אינסטלציה
מוסכים
שרותי מנעולנים


In [3]:
# Filling missing Scores based on the mean of the column so that the average will not affected
# After all the values in the score columns are integers we convert the column type

filenames = Get_All_CSV_Names()

for csv in filenames:
    df = pd.read_csv(f"data/{csv}.csv")

    df_temp = df[df["Overall Score"].astype(str).str.isdigit()]
    overall_score_mean = round(pd.to_numeric(df_temp["Overall Score"]).mean())
    
    df_temp = df[df["Quality Score"].astype(str).str.isdigit()]
    quality_score_mean = round(pd.to_numeric(df_temp["Quality Score"]).mean())
    
    df_temp = df[df["Price Score"].astype(str).str.isdigit()]
    price_score_mean = round(pd.to_numeric(df_temp["Price Score"]).mean())

    df_temp = df[df["Time Score"].astype(str).str.isdigit()]
    time_score_mean = round(pd.to_numeric(df_temp["Time Score"]).mean())

    df_temp = df[df["Treatment Score"].astype(str).str.isdigit()]
    treatment_score_mean = round(pd.to_numeric(df_temp["Treatment Score"]).mean())

    for i, row in df.iterrows():

        if '-' in str(row["Overall Score"]):
            df.at[i, "Overall Score"]  = overall_score_mean
        if '-' in str(row["Quality Score"]):
            df.at[i, "Quality Score"]  = overall_score_mean
        if '-' in str(row["Price Score"]):
            df.at[i, "Price Score"]  = price_score_mean
        if '-' in str(row["Time Score"]):
            df.at[i, "Time Score"]  = time_score_mean
        if '-' in str(row["Treatment Score"]):
            df.at[i, "Treatment Score"]  = treatment_score_mean
    
    df["Overall Score"] = pd.to_numeric(df["Overall Score"])
    df["Quality Score"] = pd.to_numeric(df["Quality Score"])
    df["Price Score"] = pd.to_numeric(df["Price Score"])
    df["Time Score"] = pd.to_numeric(df["Time Score"])
    df["Treatment Score"] = pd.to_numeric(df["Treatment Score"])
    
    Save_CSV(f"{csv}.CSV", df)



In [ ]:
# Creating a new CSV for each category that includes the total number of reviews per year

categories_csv = Get_All_CSV_Names()
row = 0
index = 0
for cat in categories_csv:
    df = pd.read_csv(f"data/{cat}.csv")
    
    df_dict = pd.DataFrame.from_dict(df["Year"].value_counts().to_dict(), orient="index", columns=["Total Reviews"])
    df_dict.reset_index(inplace=True)
    df_dict.rename(columns={'index':'Year'}, inplace = True)
    df_dict.sort_values(by="Year", inplace=True)
    df_dict['Year'] = df_dict["Year"].astype(str)
   
    if index == 2:
        row = 1
        index = 0
    else:
        index+=1
    
    Save_CSV(f"/ML/summery_{cat}.csv", df_dict)